In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4MptKsSsONT0fnQsIDc2")
project = rf.workspace("deslizamentos").project("landslide-detection-augmentation")
version = project.version(12)
dataset = version.download("yolov11")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Source folder (in Colab)
source_folder = "/content/Landslide-Detection-Augmentation-12"

# Destination folder (in Google Drive)
destination_folder = "/content/drive/MyDrive/Landslide_Dataset"

# Move the folder
shutil.move(source_folder, destination_folder)


'/content/drive/MyDrive/Landslide_Dataset'

In [ ]:
import albumentations as A
import cv2
import os
import numpy as np

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.4),
    A.VerticalFlip(p=0.2),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.2))

def read_label(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            if line.strip():
                class_id, x, y, w, h = map(float, line.strip().split())
                labels.append([class_id, x, y, w, h])
    return np.array(labels)

def save_label(label_path, labels):
    with open(label_path, 'w') as file:
        for label in labels:
            file.write(' '.join(map(str, label)) + '\n')

def augment_and_save(image_path, label_path, save_image_path, save_label_path, transform, num_augmented):
    image = cv2.imread(image_path)
    labels = read_label(label_path)

    if labels.size == 0:  # Skip augmentation if no labels are found
        return

    class_labels = labels[:, 0]
    bboxes = labels[:, 1:]

    for i in range(num_augmented):
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = transformed['image']
        augmented_bboxes = transformed['bboxes']

        if len(augmented_bboxes) == 0:  # Skip if no bounding boxes remain after augmentation
            continue

        augmented_labels = np.hstack((np.array(transformed['class_labels'])[:, np.newaxis], augmented_bboxes))

        augmented_image_path = os.path.join(save_image_path, f"aug_{i}_{os.path.basename(image_path)}")
        augmented_label_path = os.path.join(save_label_path, f"aug_{i}_{os.path.basename(label_path)}")

        cv2.imwrite(augmented_image_path, augmented_image)
        save_label(augmented_label_path, augmented_labels)

# Paths to your dataset images and labels
dataset_image_path = "/content/drive/MyDrive/Landslide_Dataset/images"
dataset_label_path = "/content/drive/MyDrive/Landslide_Dataset/labels"
augmented_image_path = "/content/drive/MyDrive/Landslide_Dataset/augt/images"
augmented_label_path = "/content/drive/MyDrive/Landslide_Dataset/augt/labels"

os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Augment each image and its corresponding label in the dataset
for image_name in os.listdir(dataset_image_path):
    image_path = os.path.join(dataset_image_path, image_name)
    label_path = os.path.join(dataset_label_path, image_name.replace('.jpg', '.txt'))
    augment_and_save(image_path, label_path, augmented_image_path, augmented_label_path, transform, num_augmented=5)

In [ ]:
import shutil
import os

# Ensure the augmented directories exist
os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Copy original images to the augmented images directory
for image_name in os.listdir(dataset_image_path):
    src_image_path = os.path.join(dataset_image_path, image_name)
    dst_image_path = os.path.join(augmented_image_path, image_name)
    shutil.copy(src_image_path, dst_image_path)

# Copy original labels to the augmented labels directory
for label_name in os.listdir(dataset_label_path):
    src_label_path = os.path.join(dataset_label_path, label_name)
    dst_label_path = os.path.join(augmented_label_path, label_name)
    shutil.copy(src_label_path, dst_label_path)

In [ ]:
import os
import cv2
import numpy as np
import shutil

def main():
    # Define folder paths
    image_folder = "/content/drive/MyDrive/Landslide_Dataset/augt/images"                 # Folder with original images
    label_folder = "/content/drive/MyDrive/Landslide_Dataset/augt/labels"                   # Folder with YOLO label files
    output_image_folder = "/content/drive/MyDrive/Landslide_Dataset/nor/images"     # Folder to save normalized images
    output_label_folder = "/content/drive/MyDrive/Landslide_Dataset/nor/labels"       # Folder to copy label files

    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    # Get list of image files (supports .jpg, .jpeg, .png)
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error reading image: {image_path}")
            continue

        # Normalize image pixel values to [0,1]
        image = image.astype(np.float32) / 255.0

        # Convert normalized image back to 0-255 for saving (for visualization)
        normalized_image = (image * 255).astype(np.uint8)
        out_image_path = os.path.join(output_image_folder, image_file)
        cv2.imwrite(out_image_path, normalized_image)

        # Copy the corresponding label file if it exists (labels are in YOLO format and already normalized)
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(label_folder, label_file)
        if os.path.exists(label_path):
            out_label_path = os.path.join(output_label_folder, label_file)
            shutil.copy(label_path, out_label_path)
            print(f"Processed image and copied label: {image_file}")
        else:
            print(f"No label file for image: {image_file}")

if __name__ == '__main__':
    main()

In [ ]:
import os
import shutil
import random

def main():
    # Define source paths
    dataset_image_path = "/content/drive/MyDrive/Landslide_Dataset/normalized/images"
    dataset_label_path = "/content/drive/MyDrive/Landslide_Dataset/normalized/labels"

    # Define destination paths for images and labels
    train_image_path = "/content/drive/MyDrive/Landslide_Dataset/train/images/"
    test_image_path = "/content/drive/MyDrive/Landslide_Dataset/test/images/"
    val_image_path = "/content/drive/MyDrive/Landslide_Dataset/valid/images/"

    train_label_path = "/content/drive/MyDrive/Landslide_Dataset/train/labels/"
    test_label_path = "/content/drive/MyDrive/Landslide_Dataset/test/labels/"
    val_label_path = "/content/drive/MyDrive/Landslide_Dataset/valid/labels/"

    # Create destination directories for images and labels
    os.makedirs(train_image_path, exist_ok=True)
    os.makedirs(test_image_path, exist_ok=True)
    os.makedirs(val_image_path, exist_ok=True)
    os.makedirs(train_label_path, exist_ok=True)
    os.makedirs(test_label_path, exist_ok=True)
    os.makedirs(val_label_path, exist_ok=True)

    # Get all image files (supports .jpg, .jpeg, .png)
    images = [f for f in os.listdir(dataset_image_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    total_images = len(images)
    train_split = int(0.8 * total_images)
    test_split = int(0.9 * total_images)  # 80% train, next 10% test, remaining 10% validation

    train_images = images[:train_split]
    test_images = images[train_split:test_split]
    val_images = images[test_split:]

    # Move image and corresponding label files for training dataset
    for image_file in train_images:
        # Move image
        src_image = os.path.join(dataset_image_path, image_file)
        dst_image = os.path.join(train_image_path, image_file)
        shutil.move(src_image, dst_image)

        # Build corresponding label file name (assumed to be .txt)
        label_file = os.path.splitext(image_file)[0] + ".txt"
        src_label = os.path.join(dataset_label_path, label_file)
        dst_label = os.path.join(train_label_path, label_file)
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
        else:
            print(f"Warning: No label file found for training image: {image_file}")

    # Move image and corresponding label files for testing dataset
    for image_file in test_images:
        src_image = os.path.join(dataset_image_path, image_file)
        dst_image = os.path.join(test_image_path, image_file)
        shutil.move(src_image, dst_image)

        label_file = os.path.splitext(image_file)[0] + ".txt"
        src_label = os.path.join(dataset_label_path, label_file)
        dst_label = os.path.join(test_label_path, label_file)
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
        else:
            print(f"Warning: No label file found for testing image: {image_file}")

    # Move image and corresponding label files for validation dataset
    for image_file in val_images:
        src_image = os.path.join(dataset_image_path, image_file)
        dst_image = os.path.join(val_image_path, image_file)
        shutil.move(src_image, dst_image)

        label_file = os.path.splitext(image_file)[0] + ".txt"
        src_label = os.path.join(dataset_label_path, label_file)
        dst_label = os.path.join(val_label_path, label_file)
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
        else:
            print(f"Warning: No label file found for validation image: {image_file}")

    print(f"Moved {len(train_images)} images and their corresponding labels to training set.")
    print(f"Moved {len(test_images)} images and their corresponding labels to testing set.")
    print(f"Moved {len(val_images)} images and their corresponding labels to validation set.")

if __name__ == "__main__":
    main()

In [ ]:
import yaml
data = {
    'train': "/content/drive/MyDrive/Landslide_Dataset/train/images",
    'val': "/content/drive/MyDrive/Landslide_Dataset/valid/images",
    'test': "/content/drive/MyDrive/Landslide_Dataset/test/images",
    'nc': 2,
    'names': ['Fissure', 'Landslide']
}
with open("/content/drive/MyDrive/Landslide_Dataset/data.yaml", 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

In [ ]:
import os

def count_files(directory, extensions):
    """
    Count the number of files in `directory` that have one of the specified `extensions`.
    """
    return len([f for f in os.listdir(directory)
                if os.path.splitext(f)[1].lower() in extensions])

def main():
    # Specify the directories for images and labels
    image_folder = "/content/drive/MyDrive/Landslide_Dataset/normalized/images"  # Change as required
    label_folder = "/content/drive/MyDrive/Landslide_Dataset/normalized/labels/"    # Change as required

    # Define common file extensions for images and labels
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'}
    label_extensions = {'.txt'}  # Assuming labels are stored as .txt files

    # Count image and label files
    num_images = count_files(image_folder, image_extensions)
    num_labels = count_files(label_folder, label_extensions)

    print("Number of images:", num_images)
    print("Number of label files:", num_labels)

    # Optionally, if you want to report if any image doesn't have a corresponding label file,
    # you can do so by checking matching file names (without extensions).
    image_names = {os.path.splitext(f)[0] for f in os.listdir(image_folder)
                   if os.path.splitext(f)[1].lower() in image_extensions}
    label_names = {os.path.splitext(f)[0] for f in os.listdir(label_folder)
                   if os.path.splitext(f)[1].lower() in label_extensions}

    missing_labels = image_names - label_names
    missing_images = label_names - image_names

    if missing_labels:
        print("\nImages missing corresponding label files:")
        for name in missing_labels:
            print(" -", name)
    else:
        print("\nAll images have corresponding label files.")

    if missing_images:
        print("\nLabel files without corresponding images:")
        for name in missing_images:
            print(" -", name)
    else:
        print("\nAll label files have corresponding images.")

if __name__ == "__main__":
    main()